<a href="https://colab.research.google.com/github/ErikZA/Adventch-cliente/blob/master/appSus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
# Set dependencies
import requests 
from requests.auth import HTTPBasicAuth
import pandas as pd
import json

In [116]:
# Set up the request parameters
url = 'https://imunizacao-es.saude.gov.br/'
firstPageParam = '_search?scroll=1m'
nextPageParam = '_search/scroll'
headers = {'Content-Type': 'application/json'}
auth = HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi')
data = {'size': 10000}

In [117]:
# store data and pageId
data_store = []
pageId = ""

In [118]:
# def function to validate if is there more data do fetch
def checkDataSize(data):
  return len(data) > 0

In [119]:
# def function to store data from api
def storeDataFromApi(vc_store, vcData):
  vc_store.extend(vcData)

In [120]:
# def function to get all data from api and store it
def getDataFromApi(resData):
  vacinasOpt = resData.json()
  paginationId = vacinasOpt['_scroll_id']
  vacinasData = vacinasOpt['hits']['hits']
  return (vacinasData, paginationId)

In [125]:
# def function to fetch data with pagination
def getVaccinesWithScroll(scrollId):
  data = { "scroll_id": scrollId, "scroll": "1m" }
  print(data)
  response = requests.post(url+nextPageParam, headers=headers, auth=auth, data=json.dumps(data))
  teste = response.json()
  print('jesus')
  print(teste['_scroll_id'])
  print(teste['hits']['hits'][0]['_source']['paciente_id'])
  return response

In [126]:
# Send the request using HTTP Basic Auth
response = requests.post(url+firstPageParam, headers=headers, auth=auth, data=json.dumps(data))

In [127]:
# get data form the first page
currentData, paginationId = getDataFromApi(response)
pageId = paginationId
data_store.extend(currentData)

In [ ]:
if len(currentData) > 0:
  while True:
    dataOpt = getVaccinesWithScroll(pageId)
    currentData, paginationId = getDataFromApi(dataOpt)
    pageId = paginationId
    data_store.extend(currentData)
    if checkDataSize(currentData) == False || len(data_store) == 100000  :
      break
    else:
      continue

In [ ]:
#create a data frame with pandas
vacinasOpt = vc_store.json()
df_vacinasOpt = pd.json_normalize(vacinasOpt['hits']['hits'])
df_vacinasOpt